In [ ]:
!pip install crewai-tools==0.25.0
!pip install crewai==0.86.0
!pip install langchain_community==0.0.29

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# **Simple Blog**

In [ ]:
from crewai import LLM
llm = LLM(
    api_key="your api key",
    model="grok-4-0709"
)

## Agents

In [ ]:
from crewai import Agent, Task, Crew
planner = Agent(
    llm=llm,
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)
writer = Agent(
    llm=llm,
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements."
              "Write the blog in a well-readable manner, partitioning it into sections as per the requirement. Ensure there are at least 5-6 sections, making the entire blog lengthier.",
    allow_delegation=False,
    verbose=True
)
editor = Agent(
    llm=llm,
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Tasks

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

Define Crew

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=True
)

Kick off

In [ ]:
result = crew.kickoff(inputs={"topic": "Write a blog about us politcs"})

Display

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)

# **Simple HTML webpage**

Define Custom Tools

In [ ]:
from crewai.tools import BaseTool
from typing import Type
from pydantic import BaseModel, Field

Browse Google Image URLs

In [ ]:
!pip install serpapi==0.1.5

TOOL 1: Browse Google Images

In [ ]:
import serpapi
class GetImagesInput(BaseModel):
    """Image keyword to search for getting image urls from the web"""
    image_keyword: str = Field(..., description="Image keyword to search for getting image urls from the web")

class GetImagesFromWeb(BaseTool):
    name: str = "Get_relevant_image_urls"
    description: str = "Search for the relevant image urls on web based on the given keyword"
    args_schema: Type[BaseModel] = GetImagesInput

    def _run(self, image_keyword: str) -> list:
        print("GetImagesFromWeb tool called")
        urlList = []
        try:
          params = {
            "q": str(image_keyword),
            "engine": "google_images",
            "ijn": "0",
            "api_key": "7002b372c5ebe29de686b784ece0cde6f066b1b41b163c4d4"
          }
          search = serpapi.search(params)
          for eachObject in search['images_results'][:10]:
            urlList.append(eachObject['original'])
          print("urlList is")
          print(urlList)
          return urlList
        except Exception as e:
          return urlList
get_image_urls_tool = GetImagesFromWeb()

TOOL 2: Save HTML file tool

In [ ]:
class SaveHtmlFileInput(BaseModel):
    """Input schema for saving an HTML file."""
    html_page_content: str = Field(..., description="HTML file contents to save.")

class SaveHtmlFileTool(BaseTool):
    name: str = "Save_html_file"
    description: str = "Saves the given HTML content to a file named 'mywebpage.html'."
    args_schema: Type[BaseModel] = SaveHtmlFileInput

    def _run(self, html_page_content: str) -> str:
        filename = "mywebpage.html"
        try:
          with open(filename, "w", encoding="utf-8") as file:
              file.write(html_page_content)
          return f"File saved successfully as {filename}"
        except Exception as e:
            return f"Error saving file: {str(e)}"
save_file_tool = SaveHtmlFileTool()

Web Developer Agent

In [ ]:
web_developer = Agent(
    llm=llm,
    role = "Senior Web developer",
    goal = "Compile the results into a HTML file.",
    backstory = "You are a competent Web developer writing HTML and CSS with decades of experience. "
                "You receives a blog post from the editor "
                "Your Goal is the convert the edited final draft of the blog post in a well structured HTML file "
                "The web page should be desgined beutifully using js or css"
                "You must include images(use image urls from web) and background colores which are relevant to the topic and content mentioned in the input file."
                "Pass short relevant keywords like 'indian cricket','America','Machine learning'..etc to the given `Get_relevant_image_urls` tool fr getting relevant image urls from web."
                "Dont pass long keywords or sentences to the `Get_relevant_image_urls` tool"
                "The output will be a single HTML file all the images,fonts, css and js should be embedded inside that file"
                "The webpage must be colorfull and attractive. Add background color and font color based on the context of the blog. But dont keep it as default"
                "Save the HTML file in the folder with name current_webpage.html use given `Save_html_file` tool for saving.",
    tools = [save_file_tool,get_image_urls_tool],
    allow_delegation=False,
    verbose=True
)

Web Developer Task

In [ ]:
webpage_creation = Task(
    description=("Convert the given edited final draft of the blog post into a html file"),
    expected_output="A well-desgined HTML page which contain both images and texts and looks very pretty"
                    "The output will be a single HTML file all the images,fonts, css and js should be embedded inside that file. Pass short relevant keywords to the given tool and use given tool for getting relevant image urls from web."
                    "The webpage must be colorfull and attractive. Add background color and font color based on the context of the blog. But dont keep it as default"
                    "Save the HTML file in the folder with name current_webpage.html use python tool for saving.",

    agent=web_developer
)

Redefine the Crew

In [ ]:
crew = Crew(
    agents=[planner, writer, editor,web_developer],
    tasks=[plan, write, edit,webpage_creation],
    verbose=True
)

Kick Off

In [ ]:
result = crew.kickoff(inputs={"topic": "Malayam Cinema History"})

In [ ]:
!pip freeze